# Canada Census 2021 Data: Exploratory Data Analysis (EDA)

This notebook explores, cleans, and analyzes the Canada census 2021 dataset. We address data quality, filtering, and extract key insights about the demographic and socio-economic characteristics of Toronto's population.

In [ ]:
import pandas as pd

Because the census dataset is very large, we want to load only the parts that are relevant to our analysis—specifically, the data related to Toronto. The `census_2021_geo` file contains the geographic names (`Geo Name`) and their corresponding line numbers in the main census dataset. By using these line numbers, we can efficiently read only the rows from the main census file that correspond to Toronto regions, significantly reducing memory usage and processing time. This targeted loading approach allows us to focus our analysis on the Forward Sortation Areas (FSAs) within Toronto without handling the entire national dataset.

In [ ]:
census_geo_path = "../data/00_raw/census_2021_geo.csv"
df_geo = pd.read_csv(census_geo_path)

display(df_geo.head())

The Geo Name values related to Toronto are started with "M" (for example, "M1A", "M2B", etc.). We will filter the `census_2021_geo` file to extract only those rows that correspond to Toronto's FSAs.

In [ ]:
# Find the start and end line numbers for Toronto FSAs (Geo Name starts with "M")
start_line = df_geo[df_geo["Geo Name"].str.startswith("M")].iloc[0]["Line Number"]
end_line = df_geo[df_geo["Geo Name"].str.startswith("N")].iloc[0]["Line Number"]

print(f"Start line for Toronto FSAs: {start_line}")
print(f"End line for Toronto FSAs: {end_line}")

# Calculate skiprows and nrows for efficient CSV loading
nskiprows = start_line - 1  # skip header and lines before Toronto
nrows = end_line - start_line  # number of Toronto rows

print(f"nskiprows: {nskiprows}")
print(f"nrows: {nrows}")

In [ ]:
census_data_path = "../data/00_raw/census_2021.csv"
df_census = pd.read_csv(
    census_data_path,
    header=0,
    encoding="latin1",
    skiprows=range(1, nskiprows),
    nrows=nrows,
)
print("--- Head ---")
display(df_census)
print("\n--- Info ---")
df_census.info()
print("\n--- Describe ---")
display(df_census.describe(include="all"))

In [ ]:
# check if ALT_GEO_CODE and GEO_NAME are the same
df_census["ALT_GEO_CODE"] = df_census["ALT_GEO_CODE"].astype(str)
df_census["GEO_NAME"] = df_census["GEO_NAME"].astype(str)
print("\n--- Check ALT_GEO_CODE and GEO_NAME ---")
df_census["check"] = df_census["ALT_GEO_CODE"] == df_census["GEO_NAME"]
if df_census["check"].all():
    print("All ALT_GEO_CODE and GEO_NAME match.")

### Data Cleaning and Preparation Strategy

The initial exploration of the `df_census` DataFrame reveals several areas that need attention before it can be effectively used for analysis and merging with other datasets. The primary goals of this cleaning process are to:
1.  Ensure correct and meaningful column names.
2.  Optimize data types for memory efficiency and appropriate representation.
3.  Understand and prepare for handling missing values, which often have specific meanings in this dataset.
4.  Prepare the data for potential pivoting operations.

Here's a breakdown of the planned steps:

1.  **Address Column Naming (SYMBOL columns):**
    *   **Issue:** The dataset contains columns like `SYMBOL`, `SYMBOL.1`, `SYMBOL.2`, etc. These arise because the original CSV has multiple columns named "SYMBOL", and pandas automatically renames duplicates upon import.
    *   **Solution:** These `SYMBOL` columns provide crucial context (e.g., data suppression, reliability, applicability) for their corresponding data columns (e.g., `C1_COUNT_TOTAL`, `C2_COUNT_MEN+`). They need to be renamed to establish an explicit link. For example:
        *   The first `SYMBOL` column (original index 12, next to `C1_COUNT_TOTAL`) will be renamed to `C1_SYMBOL`.
        *   `SYMBOL.1` (original index 14, next to `C2_COUNT_MEN+`) will be renamed to `C2_SYMBOL`.
        *   This pattern will be applied systematically for all `SYMBOL.n` columns, linking them to `C3_...`, `C4_...` up to `C18_...`.
    *   **Benefit:** This renaming will make it significantly easier to interpret data quality and understand missingness for each specific metric.

2.  **Optimize Data Types:**
    *   **Goal:** Improve memory usage and ensure data is represented in the most appropriate format for analysis.
    *   **Categorical Columns:**
        *   `DGUID`: While `object` is acceptable, given its role as a geographic identifier that will repeat for different characteristics, converting to `category` can be beneficial if memory becomes a concern. For joining purposes, it should be treated as a string.
        *   `ALT_GEO_CODE`: This is the primary key for joining with other datasets (like auto theft data). It should be cleaned (e.g., trim whitespace, ensure consistent casing) and then converted to `category` or kept as `object` (string).
        *   `CHARACTERISTIC_ID`: This identifier for census characteristics should be `category`.
        *   `CHARACTERISTIC_NAME`: Contains many repeated textual descriptions of census variables. Converting to `category` is highly recommended for memory savings and performance.
        *   `DATA_QUALITY_FLAG`: Currently `int64`. If these are discrete codes (as suggested by the `describe` output showing a limited range and integer-like values), converting to `category` is appropriate after verifying unique values.
        *   All `SYMBOL` columns (after renaming): These contain codes like 'x', '..', 'F'. They should be converted to `category`.
    *   **Numeric Columns:**
        *   `TNR_SF`, `TNR_LF`: Currently `float64`. These likely represent "Total Number of Records" for short and long forms. If they are always whole numbers, they should be converted to a nullable integer type like `pd.Int16Dtype()` or `pd.Int32Dtype()` after handling NaNs, to save memory and accurately reflect their nature as counts.
        *   `CHARACTERISTIC_NOTE`: Currently `float64` with many NaNs. These are likely numerical IDs for footnotes. Convert to a nullable integer type (e.g., `pd.Int16Dtype()`) to handle NaNs appropriately.
        *   Data Columns (`C1_COUNT_TOTAL` to `C18_RATE_HI_CI_WOMEN+`): `float64` is generally suitable for these columns as they can represent counts, rates, and confidence intervals, which may include decimal values and will contain NaNs where data is suppressed or not applicable. However, `float32` could be considered if precision allows, to save memory.
    *   **Columns to Drop:**
        *   `CENSUS_YEAR`: Appears to be constant (2021). Can be dropped if this information is stored elsewhere or considered implicit.
        *   `GEO_LEVEL`: Appears to be constant ("Forward sortation area"). Can be dropped.
        *   `GEO_NAME`: Appears to be identical to `ALT_GEO_CODE` in the sample. If this holds true for the entire dataset, one of them can be dropped to avoid redundancy. `ALT_GEO_CODE` is preferred as it's a more standard geographic code.

3.  **Hierarchical Characteristic Names:**
    *   **Observation:** The `CHARACTERISTIC_NAME` column has leading spaces, which seem to indicate a hierarchical structure (e.g., "Total...", "  Male...", "    Aged 15-24...").
    *   **Action:**
        *   Create a new column, `CHARACTERISTIC_LEVEL`, by counting the leading spaces (divided by 2, assuming 2 spaces per indent level) to quantify the hierarchy.
        *   Strip leading/trailing whitespace from `CHARACTERISTIC_NAME` itself for cleaner values.

By implementing these cleaning steps, the dataset will be more robust, memory-efficient, and easier to work with for subsequent analysis and merging tasks. The explicit linking of `SYMBOL` columns to their data columns is particularly crucial for correctly interpreting the data's meaning and quality.

In [ ]:
column_dtypes = {
    "DGUID": "category",
    "ALT_GEO_CODE": "object",
    "CHARACTERISTIC_ID": "category",
    "CHARACTERISTIC_NAME": "object",
    "CHARACTERISTIC_NOTE": "Int16",
    "DATA_QUALITY_FLAG": "category",
    "TNR_SF": "float32",
    "TNR_LF": "float32",
}


original_columns = pd.read_csv(census_data_path, nrows=0, encoding="latin1").columns.tolist()
new_columns = []
for col in original_columns:
    if col == "SYMBOL":
        new_columns.append("C1_SYMBOL")
        column_dtypes["C1_SYMBOL"] = "category"
    elif col.startswith("SYMBOL."):
        idx = int(col.split(".")[1])
        new_columns.append(f"C{idx+1}_SYMBOL")
        column_dtypes[f"C{idx+1}_SYMBOL"] = "category"
    else:
        new_columns.append(col)


columns_to_drop = [
    "CENSUS_YEAR",
    "GEO_LEVEL",
    "GEO_NAME",
]

df_census = pd.read_csv(
    census_data_path,
    header=0,
    encoding="latin1",
    skiprows=range(1, 1450232 + 1),
    nrows=157526,
    names=new_columns,
    usecols=lambda x: x not in columns_to_drop,
    dtype=column_dtypes,
)

# 
df_census['CHARACTERISTIC_LEVEL'] = df_census['CHARACTERISTIC_NAME'].apply(lambda x: int((len(x) - len(x.lstrip(' ')))/2))
df_census['CHARACTERISTIC_NAME'] = df_census['CHARACTERISTIC_NAME'].str.strip().astype("category")


# Display the first few rows with new column names and info to verify
print("--- Head --")
display(df_census.head())
print("\n--- Info ---")
df_census.info()
print("\n--- Describe ---")
display(df_census.describe(include="all").T)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.heatmap(df_census.isnull(), cbar=False, yticklabels=False)
plt.title("Missing Values Heatmap")
plt.show()

### Understanding and Handling Missing Values in the Census Dataset

Missing values in this dataset are not always simply "missing"; They often carry specific meanings, especially as indicated by the various `SYMBOL` columns.

- `CHARACTERISTIC_NOTE`
  - Many NaNs in `CHARACTERISTIC_NOTE` are expected; not every characteristic has a footnote.
  - NaN here usually means "no note" and can generally be left as is.

#### Data Columns (`C1` to `C18`) and Their SYMBOL Columns

- **Link Between Data and SYMBOL Columns:**  
    A NaN in a data column (e.g., `C1_COUNT_TOTAL`) often corresponds to a value in its paired SYMBOL column (e.g., `C1_SYMBOL` might be `'x'`, `'F'`, `'...'`, or `'..'`).

    - `x`: Suppressed for confidentiality.
    - `F`: Too unreliable to be published.
    - `...`: Not applicable.
    - `..`: Not available.

- **SYMBOL Column NaNs:**  
    If a SYMBOL column is NaN, it typically means the corresponding data value is present and valid (not suppressed or inapplicable).
- For example, if `C1_COUNT_TOTAL` is NaN and `C1_SYMBOL` is `'x'`, the data was suppressed. Replacing with 0 would be incorrect.

#### Confidence Interval (CI) Columns

- CI columns often have more NaNs than main estimates. This is normal, as CIs may not be provided for small or unreliable estimates.

#### Strategy After Pivoting

- **Suppressed or Unreliable Data (`x`, `F`):**  
    These are intentionally missing. We may keep them as NaN and note their status, or retain the symbol if our analysis/model can handle categorical values. Imputation is generally not recommended.
- **Not Applicable (`...`):**  
    For count data, this might sometimes be replaced with 0, but for rates, interpretation is more complex.
- **Not Available (`..`):**  
    These are truly missing values.


In [ ]:
df_census['CHARACTERISTIC_NAME'][df_census['CHARACTERISTIC_LEVEL'] == 1].unique().tolist()

Depending on the detail and complexity of the analysis, we may choose to filter the dataset based on CHARACTERISTIC_LEVEL. for example CHARACTERISTIC_LEVEL larger than 4 are typically more granular and may not be necessary for high-level analysis.

In [ ]:
df_census = df_census['CHARACTERISTIC_NAME'][df_census['CHARACTERISTIC_LEVEL'] < 5]

print("\n--- Filtered CHARACTERISTIC_NAME ---")
display(df_census)